In [60]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import datetime, os
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from numpy import linalg as LA
import cv2
from tqdm import tqdm

In [9]:
# Create atlas critisations for specific gender
# Param : gender
#       - male
#       - female

def create_table(gender):
    atlas_month_norms = dict()
    for month in range(0,40):
        try:
            fname = f'./Atlas_of_Hand_Bone_Age_Fixed/{gender}-{month}m.jpg'
            arr = cv2.imread(fname)
            temp = {f"Month-{month}":LA.norm(arr) }
            atlas_month_norms.update(temp)
        except Exception as e:
            print(f"{e} and {month}")
            
    atlas_age_norms = dict()
    for age in range(4,19):
        fname = f'./Atlas_of_Hand_Bone_Age_Fixed/{gender}-{age}y.jpg'
        arr = cv2.imread(fname)
        temp = {f"Month-{age*12}":LA.norm(arr) }
        atlas_age_norms.update(temp)
        
    return {**atlas_month_norms, **atlas_age_norms}

In [10]:
# Get atlas GP

def get_atlas():
    #loading dataframes
    train_df = pd.read_csv('./atlas.csv')

    #appending file extension to id column for both training and testing dataframes
    train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.jpg')
    
    return train_df

In [11]:
# Get dataset RNSA

def get_dataset():
    #loading dataframes
    train_df = pd.read_csv('./input/boneage-training-dataset.csv')
    test_df = pd.read_csv('./input/boneage-test-dataset.csv')

    #appending file extension to id column for both training and testing dataframes
    train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.png')
    test_df['Case ID'] = test_df['Case ID'].apply(lambda x: str(x)+'.png') 
    train_df['gender'] = train_df['male'].apply(lambda x: 'male' if x else 'female')
    return train_df, test_df

In [44]:
# Compare each Xray with the set of images of Atlas and predict

def predict(train_data_array, atlas):
    norm_of_train_data = LA.norm(train_data_array)
    #print(norm_of_train_data)
    temp = []
    for k,cathegory in enumerate(atlas.values()):
        temp.append(abs(cathegory - norm_of_train_data))
    #print((temp))
    #print(min(temp))
    minimum_diff = min(temp)
    index = temp.index(minimum_diff)
    #print(index)
    return list(atlas.keys())[index]

In [13]:
# By means of OpenCV, change the brightness of the images

def change_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,value)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [21]:
# Returns the number of gender per each

def count_by_gender(gender, train_data):
    return (train_data['gender'].value_counts())[gender]

In [16]:
gp_atlas = get_atlas()
train_df,_ = get_dataset()
atlas_male = create_table("male")
atlas_female = create_table("female")

unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 0
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 1
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 2
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 3
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 4
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 5
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 6
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 7
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 9
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 11
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 13
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 15
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 17
unsupported operand type(s) for *: 'NoneType' and 'NoneType' and 19
unsupported operand type(s) for *: 'NoneType' and 'NoneTy

In [75]:
male_stats = np.zeros(count_by_gender('male', train_df) * 3).reshape(count_by_gender('male', train_df),3)

In [83]:
#for male

acc = 0
count = 0

train_male_df = train_df[train_df['gender'] == "male"]

for filename, boneage, gender in tqdm(list(train_male_df[['id','boneage','gender']].values)):

    fname = './input/boneage-training-dataset/boneage-training-dataset/'+ filename
    img = cv2.imread(fname)
    img = cv2.resize(img,(691,691))
    arr = change_brightness(img,30)
    predict_month = int(predict(arr, atlas_male).split('-')[-1])
    status = False
    

    if predict_month <= (boneage + 6) and predict_month >= (boneage - 6):
        acc += 1
        status = True
    male_stats[count, 0] = boneage
    male_stats[count, 1] = predict_month
    male_stats[count, 2] = status
    #plt.imshow(arr, cmap='gray', vmin=0, vmax=255)
    #plt.title('Image name:{}  Bone age: {} years  Gender: {} Predict {}'.format(filename, round(boneage/12, 2), gender, predict_month))
    #plt.axis('off')
    #plt.show()
    count += 1
        
stats_male_df = pd.DataFrame(male_stats, columns = ['Actual', 'Predicted', 'Status'])
acc = (acc / count_by_gender('male', train_df) ) * 100
print(stats_male_df.head())
print(acc)

100%|██████████| 6833/6833 [13:05<00:00,  8.70it/s]

   Actual  Predicted  Status
0   120.0      132.0     0.0
1   138.0       48.0     0.0
2   150.0       48.0     0.0
3   156.0       48.0     0.0
4    36.0      180.0     0.0
4.917313039660471


In [84]:
female_stats = np.zeros(count_by_gender('female', train_df) * 3).reshape(count_by_gender('female', train_df),3)

In [86]:
#for female

acc = 0
count = 0

train_female_df = train_df[train_df['gender'] == "female"]

for filename, boneage, gender in tqdm(list(train_female_df[['id','boneage','gender']].values)):

    fname = './input/boneage-training-dataset/boneage-training-dataset/'+ filename
    img = cv2.imread(fname)
    img = cv2.resize(img,(691,691))
    arr = change_brightness(img,30)
    predict_month = int(predict(arr, atlas_female).split('-')[-1])
    status = False
    

    if predict_month <= (boneage + 6) and predict_month >= (boneage - 6):
        acc += 1
        status = True
    female_stats[count, 0] = boneage
    female_stats[count, 1] = predict_month
    female_stats[count, 2] = status
    #plt.imshow(arr, cmap='gray', vmin=0, vmax=255)
    #plt.title('Image name:{}  Bone age: {} years  Gender: {} Predict {}'.format(filename, round(boneage/12, 2), gender, predict_month))
    #plt.axis('off')
    #plt.show()
    count += 1
        
stats_female_df = pd.DataFrame(female_stats, columns = ['Actual', 'Predicted', 'Status'])
acc = (acc / count_by_gender('female', train_df) ) * 100
print(stats_female_df.head())
print(acc)

100%|██████████| 5778/5778 [11:32<00:00,  8.35it/s]

   Actual  Predicted  Status
0   180.0       20.0     0.0
1    12.0       20.0     0.0
2    94.0       20.0     0.0
3    82.0       20.0     0.0
4   126.0       10.0     0.0
4.101765316718588


In [88]:
stats_male_df.head(20)

,Actual,Predicted,Status
0,120.0,132.0,0.0
1,138.0,48.0,0.0
2,150.0,48.0,0.0
3,156.0,48.0,0.0
4,36.0,180.0,0.0
5,138.0,14.0,0.0
6,138.0,132.0,1.0
7,156.0,48.0,0.0
8,180.0,8.0,0.0
9,57.0,48.0,0.0


In [89]:
stats_female_df.head(20)

,Actual,Predicted,Status
0,180.0,20.0,0.0
1,12.0,20.0,0.0
2,94.0,20.0,0.0
3,82.0,20.0,0.0
4,126.0,10.0,0.0
5,132.0,20.0,0.0
6,120.0,20.0,0.0
7,188.0,20.0,0.0
8,4.0,20.0,0.0
9,36.0,20.0,0.0
